In [ ]:
# 下載檔案
!gdown 1WaKegKk4J3cf1iHghgSRmgrt0MlWIpkq -O ./

In [ ]:
# Install Requirements
!apt-get -qq install poppler-utils tesseract-ocrlibtesseract-dev tesseract-ocr-chi-sim tesseract-ocr-chi-tra tesseract-ocr-chi-sim-vert tesseract-ocr-chi-tra-vert
# Upgrade Pillow to latest version
%pip install -q --user --upgrade pillow
# Install Python Packages
%pip install -q unstructured["all-docs"]==0.16.9
# NOTE: you may also upgrade to the latest version with the command below,
#       though a more recent version of unstructured will not have been tested with this notebook
%pip install -q --upgrade unstructured

In [ ]:
## 請先點選工作列 [執行階段]-> 重新啟動工作階段
## 請先點選工作列 [編輯]-> 清除所有輸出內容
## 右側RAM/磁碟標記 -> 變更執行階段 -> T4 GPU
# Read PDF
from typing import Any
from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf
#from unstructured.partition.auto import partition
# 使用 unstructured.io 解析中文文本
#elements = partition(file=file_io, languages=["chi_sim"])

#1
raw_pdf_elements = partition_pdf(
    filename="./1Q23-EPR-with-Tables-FINAL.pdf",
    languages=["chi_tra", "chi_tra_vert","chi_sim", "chi_sim_vert"],
    #languages=["chi_tra", "chi_tra_vert","chi_sim", "chi_sim_vert"])
    extract_images_in_pdf=False,
    infer_table_structure=True,
    #include_page_breaks=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path="./im2",
)

#2
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

unique_categories = set(category_counts.keys())
category_counts

#3
class Element(BaseModel):
    type: str
    text: Any

table_elements = []
text_elements = []
all_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        text=str(element)
        text = ''.join(text.split())
        table_elements.append(Element(type="table", text=text))
        all_elements.append(Element(type="table", text=text))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        text=str(element)
        text = ''.join(text.split())
        text_elements.append(Element(type="text", text=text))
        all_elements.append(Element(type="text", text=text))

#4
print(len(table_elements))
print(len(text_elements))
print(len(all_elements))


In [ ]:
for i, table in enumerate(table_elements):
    print('cutoff', i, table)
for i, all in enumerate(all_elements):
    print('cutoff', i, all)

In [ ]:
# 印出表單
for i, table in enumerate(table_elements):
    print(i, table)

In [ ]:
# 印出段落
for i, text in enumerate(text_elements):
    print(i, text)

In [ ]:
print('將下列印出的表格一內容貼進去GPT https://chat.openai.com/')
print('')
print('You are responsible for concisely summarizing table or text chunk')
print(table_elements[1].text)

# 翻譯
I want you to act as an Chinese translator, spelling corrector and improver. I will speak to you in any language and you will detect the language, translate it and answer in the corrected and improved version of my text, in Chinese. I want you to replace my simplified A0-level words and sentences with more beautiful and elegant, upper level Chinese words and sentences. Keep the meaning same, but make them more literary. I want you to only reply the correction, the improvements and nothing else, do not write explanations.  

Translate the previous conversation